In [77]:
from iotcloudtemp.connect import revive_connection, get_thing_id, checkboxes_table, get_temp_by_hour
from datetime import timedelta
import pandas as pd

#  Pre-Load data
client_things, client_properties = revive_connection()
thing_id, properties_unflat = get_thing_id(client_things, client_properties)
properties = [item for sublist in properties_unflat for item in sublist]
df_propids, dict_propid_list = checkboxes_table(properties)

def get_data(client_properties, df_propids):
    df = pd.DataFrame({})
    for tid, tname ,pid, pname in zip(df_propids.thing_id.values, df_propids.thing_name.values, df_propids.id.values, df_propids.purpose.values):
        data_dict = client_properties.properties_v2_timeseries(tid, pid).data
        times = [el.time for el in data_dict]
        values = [el.value for el in data_dict]
        df['datetime'] = times
        df[pid] = values
        df['date'] = pd.to_datetime(df['datetime']).dt.strftime('%d.%m.%Y')
        df['time'] = pd.to_datetime(df['datetime']).dt.strftime('%H:%M')
        df['hour'] = pd.to_datetime(df['datetime']).dt.hour + pd.to_datetime(df['datetime']).dt.minute/60
        df['hour_rounded'] = round(pd.to_datetime(df['datetime']).dt.hour + pd.to_datetime(df['datetime']).dt.minute/60)

    # Make the averages
    df_avg = df.drop(['datetime', 'date', 'time', 'hour'], axis=1)\
        .groupby('hour_rounded', as_index=False).agg(['mean', 'std'])
    df_avg['hour_rounded'] = df_avg.index
    df_mean = df.drop(['datetime', 'date', 'time', 'hour'], axis=1)\
        .groupby('hour_rounded', as_index=False).agg('mean').add_suffix('_mean')
    df_std = df.drop(['datetime', 'date', 'time', 'hour'], axis=1)\
            .groupby('hour_rounded', as_index=False).agg('std').add_suffix('_std')
    df_union = pd.concat([df_mean, df_std], axis=1)
    return df, df_union

df, df_avg = get_data(client_properties, df_propids)

In [100]:
df[['eab17e2c-02bb-44c1-ba88-ce38ce214670', 'time']].tail(1)

,eab17e2c-02bb-44c1-ba88-ce38ce214670,time
999,9.090909,16:41
